# Table Valued Functions

Although sounding similar, a Table Valued Function is very different to a Table Based Scalar Function in that, instead of a single value, it returns a table as its result.

We can, for example, pass in a single employeeID and return a whole table of information such as name, salary, date of birth etc.

This can be a lot more useful than the single valued counterpart.

In a similar way to Views we can utilise a TVF (Table Valued Function) in order to restrict access to data in underlying tables by granting only access to the function itself and not the tables beneath.

And as with Views and Scalar functions, these also allow us to hide complex logic from users, giving them a centralised and reusable function which we know will return a set output time and again rather than each user having to code their own solution to a generic problem.

Finally, a last similarity to views is that, as these return a table, we can also JOIN to a TVF, although this latter trait can have some performance issues.

However, in one critical difference to a View, being a function we are allowed to pass parameters to a TVF which can make it very powerful.

Note that as with our other examples, we cannot create temporary TVFs and therefore the CREATE statements below are for reference only and will fail if executed through Jupyter against the demo database.

#### Basic TVF

For our Basic TVF we shall start with the View that we used at the end of the View section:

In [ ]:
drop view if exists hr.vwEmployeeInfo
go

create view hr.vwEmployeeInfo
as
	select e.employeeID, e.title, e.firstName, e.lastName,
			concat_ws(' ', e.title, e.firstName, e.lastName) as fullname,
			t.jobTitle, d.department
	from JupyterDatabase.hr.employee as e
	join JupyterDatabase.hr.jobTitle as t
	on e.jobTitleID = t.jobTitleID
	join JupyterDatabase.hr.department as d
	on t.departmentID = d.departmentID;
go

We wish to mimic this as a TVF, so how can we do this?

The syntax for a TVF is as follows:

<code>CREATE FUNCTION <i>schema.functionName</i><br/>
(<br/>
    <i>@parameters parameterDataType</i><br/>
)<br/>
RETURNS <i>@myTable table</i><br/>
(<br/>
    <i>columnNames columnDataTypes</i><br/>
)<br/>
AS<br/>
BEGIN<br/>
&nbsp&nbsp&nbsp&nbspINSERT INTO @myTable<br/>
&nbsp&nbsp&nbsp&nbspSELECT columns<br/>
&nbsp&nbsp&nbsp&nbspFROM table<br/>
&nbsp&nbsp&nbsp&nbspWHERE column = clause<br/>

    RETURN<br/>
END</code>

We can immediately see that this follows the same basic template as the Scalar Function with the exception that we are returning a table instead of a singular value and that return table needs to be defined as part of the function.

Therefore, using this as a template, we can switch our View to a TVF as follows:

In [ ]:
drop function if exists hr.fnEmployeeInfo
go

create function hr.fnEmployeeInfo()
returns @myTable table
(
    employeeID int,
    title varchar(10),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(61),
    jobTitle varchar(50),
    department varchar(50)
)
as
begin
    insert into @myTable
    (
        employeeID, title, firstName, lastName,
		fullName, jobTitle, department
    )
	select e.employeeID, e.title, e.firstName, e.lastName,
			concat_ws(' ', e.title, e.firstName, e.lastName) as fullname,
			t.jobTitle, d.department
	from JupyterDatabase.hr.employee as e
	join JupyterDatabase.hr.jobTitle as t
	on e.jobTitleID = t.jobTitleID
	join JupyterDatabase.hr.department as d
	on t.departmentID = d.departmentID;

	return
end
go

This may look long winded but it is not a complicated piece of syntax, rather just a little more explicit than we had to be with our View.

As we have no parameters (due to it mimicking a View) we can use it exactly as we would have used a View, the only difference being that we need brackets as it is now a function, and the brackets can be empty as we are not accepting any parameters:

In [ ]:
select *
from JupyterDatabase.hr.fnEmployeeInfo();

So why is this any different than a View?

Well, let's consider the following:

* We have been asked to only return Employee Information where the employee was born before a passed in year
* For security we cannot divulge the employee's birthday but are allowed to indicate year of birth alone

This is a little obscure, but it will server our purposes.

If we think about our View then we can begin to see that this is an awkward requirement:
* Our View does not return the date of birth and therefore we cannot reference it in a WHERE clause
* We cannot hard code a date of birth into our view as the year filter can change
* We cannot alter the View to return a date of birth as we are not allowed to show this
* We could amend the View to return Year of Birth as a column, but do not wish to return more than necessary
 * But this would cause issues with performance as we would need to return all data before we could filter

However, if we consider the TVF:
* We can pass in a year and use the parameter directly within our query
 * This allows for direct filtering on our base table and therefore is performant

This is a much simpler and cleaner solution, returning no excess columns, and performing well.

Let's look at the view first:

In [ ]:
drop view if exists hr.vwEmployeeInfoYear
go

create view hr.vwEmployeeInfoYear
as
	select e.employeeID, e.title, e.firstName, e.lastName,
			concat_ws(' ', e.title, e.firstName, e.lastName) as fullname,
			t.jobTitle, d.department,
            year(e.dob) as birthYear  -- Add the year of birth to the output list
	from JupyterDatabase.hr.employee as e
	join JupyterDatabase.hr.jobTitle as t
	on e.jobTitleID = t.jobTitleID
	join JupyterDatabase.hr.department as d
	on t.departmentID = d.departmentID;
go

In [ ]:
select *
from JupyterDatabase.hr.vwEmployeeInfoYear
where birthYear < 1960;

This is a success, but will cause performance problems (as we need to generate the entire dataset prior to looking for anything matching our filter) and has involved a change in the View output (which could cause problems to anyone using <code>select *</code> in their queries.

So now let's look at adjusting our TVF and see if this is any cleaner:

In [ ]:
drop function if exists hr.fnEmployeeInfoYear
go

create function hr.fnEmployeeInfoYear(@year date)  -- Add parameter to the function.  We will pass in a date.
returns @myTable table
(
    employeeID int,
    title varchar(10),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(61),
    jobTitle varchar(50),
    department varchar(50)
)
as
begin
    insert into @myTable
    (
        employeeID, title, firstName, lastName,
		fullName, jobTitle, department
    )
	select e.employeeID, e.title, e.firstName, e.lastName,
			concat_ws(' ', e.title, e.firstName, e.lastName) as fullname,
			t.jobTitle, d.department
	from JupyterDatabase.hr.employee as e
	join JupyterDatabase.hr.jobTitle as t
	on e.jobTitleID = t.jobTitleID
	join JupyterDatabase.hr.department as d
	on t.departmentID = d.departmentID
    where e.dob < @year;     -- Add a filter which can act on the hr.employee table directly and with good performance

	return
end
go

In [ ]:
select *
from JupyterDatabase.hr.fnEmployeeInfoYear('1960-01-01');

Our output is the same and yet with much less of an impact to any code utilising the results and whilst maintaining performance.

#### Performance Issues

Sadly, although this seems fantastic (it has good performance, can accept parameters, is centralised and reusable), it does have quite a large drawback when not used correctly.

If you choose to JOIN to your TVF then performance can be severly impacted.

Remembering back to our Table Based Scalar Functions in which we noted that every record in our results set required the Table Based Scalar Function to be re-evaluated from scratch, this is exactly the same with the TVF.

However, whereas we were in a potential performance nightmare with the Scalar Function, the issues can be even more extreme when there are JOINs involved between large tables and they are being performed possibly thousands of times.

Therefore although TVFs are good, please be very mindful when trying to join to these functions as you may find that performance is appalling and, in some cases, your queries may not return for a very long time.

#### APPLY and TVFs

One last comment in regard to joining to a TVF, you may be okay with the syntax of joining a table to a function (as the syntax is the same as we have already discussed), but wondering how we join to a TVF which requires parameters to be passed in.

This is where CROSS APPLY comes to the fore.

In order to JOIN to a function which requires parameters we need to use CROSS APPLY which will allow us to pass columns in the FROM table to the APPLY.

For example, let's create a new function which returns sales totals for our our sales people based on a passed in EmployeeID:

In [ ]:
drop function if exists sales.fnSalesPerPerson
go

create function sales.fnSalesPerPerson(@employeeID int)
returns @myTable table
(
	employeeID int,
	salesTotal decimal(10, 2)
)
as
begin
	insert into @myTable(employeeID, salesTotal)
	select salesPersonID, sum(totalDue)
	from sales.salesOrderHeader
	where salesPersonID = @employeeID
	group by salesPersonID;

	return
end
go

Let's now create a join to our hr.employee table in which we attempt to obtain sales information for three of our staff.  Note this will fail as we are using a plain JOIN, but it's a good learning experience:

In [ ]:
select e.employeeID, e.firstName, e.lastName, s.salesTotal
from JupyterDatabase.hr.employee as e
join JupyterDatabase.sales.fnSalesPerPerson(e.employeeID) as s
on e.employeeID = s.employeeID
where e.employeeID in (41, 49, 52);

Well that was expected.  So let's now try the CROSS APPLY route:

In [ ]:
select e.employeeID, e.firstName, e.lastName, s.salesTotal
from JupyterDatabase.hr.employee as e
cross apply JupyterDatabase.sales.fnSalesPerPerson(e.employeeID) as s
where e.employeeID in (41, 49, 52);

Not only does this work but we can see that we no longer need an ON clause as this is negated by the passing of the parameter to our function which acts as the ON clause for our query.